## Proyecto 4 - Sistemas de Recomendacion 

En el siguiente notebook se buscará desarrollar un mejor modelo predictivo que el logrado en el proyecto 3 para la plataforma Steam en la recomendación de juegos a los usuarios.

Se tiene acceso a dicho proyecto a través del siguiente enlace: https://github.com/Pinchoff/Proyecto_3_SR_DS_Acamica/blob/main/DS_Proyecto_03_SR.ipynb

- En primera instancia generare un recomendador simple, donde mostraremos los juegos mas populares y aclamados por los jugadores que tendran una mayor probabilidad de gustar a los usuarios promedio. Este modelo no da recomendaciones personalizadas en base al usuario.
- Luego para mejorar el arranque en frio solo dejare a usuarios que han probado mas de un juego.
- Continúo en la construcción de un recomendador basado en contenido para poder utilizarlo junto al filtro colaborativo desarrollado en el Proyecto 3 y lograr un recomendador hibrido para obtener recomendaciones mucho más personalizadas.
   

Para el presente proyecto, se buscará la implementación de un modelo NLP. Dichos modelos son elegidos debido a que en la comunidad son reconocidos como ser muy buenos para predicciones con sistemas de recomendación.

In [1]:
#Importo las librerias necesarias 

import gzip
import pandas as pd
import numpy as np
import gc
import seaborn as sns
import matplotlib.pyplot as plt

from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import GridSearchCV

from ast import literal_eval

from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import gc

#Los datos estan comprimidos
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)
        
import warnings
warnings.filterwarnings('ignore')
#plotly.tools.set_credentials_file(username='rounakbanik', api_key='xTLaHBy9MVv5szF4Pwan')

sns.set_style('whitegrid')
sns.set(font_scale=1.25)
pd.set_option('display.max_colwidth', 50)

Tenemos a disposición dos datasets, uno de gran tamaño con mas de 7000000 de reviews de los usuarios y el segundo mucho mas pequeño con 32135 juegos y sus caracteristicas.

Debido a que la potencia de cálculo que poseo es muy limitada, trabajare con una parte del dataset de gran tamaño y la mitad del dataset de juegos.

## Dataset de Reviews

Cargo las Reviews ya prepocesada en el Proyecto 3

In [70]:
data_Rev = pd.read_csv(r'D:/Acamica/Clases Acamica/Sprint 3/Proyecto/Datos_2_Review.csv')

In [20]:
data_Rev.head(2)

,calification,product_id,username
0,1,725280,Chaos Syren
1,2,328100,hello?<


## Dataset de Games

Cargo los juegos y preparo el dataset para realizar los recomendadores en especial el recomendador simple y el basado en contenido.

In [29]:
contador = 0
data_games_little = []

n = 2
for l in parse('D:/Acamica/Clases Acamica/Sprint 3/Proyecto/steam_games.json.gz'):
    if contador%n == 0:
        data_games_little.append(l)
    else:
        pass
    contador += 1

In [30]:
data_games = pd.DataFrame(data_games_little)
data_games.shape

(16068, 16)

In [31]:
data_games.shape

(16068, 16)

In [32]:
data_games.head(2)

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN


In [33]:
data_games.isna().sum()

publisher          4067
genres             1677
app_name              2
title              1057
url                   0
release_date       1066
tags                 82
discount_price    15951
reviews_url           1
specs               330
price               699
early_access          0
id                    1
developer          1689
sentiment          3531
metascore         14688
dtype: int64

In [127]:
columnas_descartables = ['title','price','discount_price','metascore','reviews_url','early_access','release_date','url','publisher','sentiment']
data_Gam = data_games.drop(columnas_descartables,1)

In [35]:
data_Gam.shape

(16068, 6)

In [36]:
data_Gam.head(2)

,genres,app_name,tags,specs,id,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],761140,Kotoshiro
1,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",670290,Poolians.com


In [37]:
data_Gam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16068 entries, 0 to 16067
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   genres     14391 non-null  object
 1   app_name   16066 non-null  object
 2   tags       15986 non-null  object
 3   specs      15738 non-null  object
 4   id         16067 non-null  object
 5   developer  14379 non-null  object
dtypes: object(6)
memory usage: 753.3+ KB


In [38]:
#Convierto la columna 'id' a enteros 
data_Gam = data_Gam[data_Gam['id'].notna()]
data_Gam['id'] = data_Gam['id'].astype(np.uint32)
#Pongo la columna 'id' como indice para luego utilizar .loc[]
data_Gam.set_index(['id'],inplace = True)
data_Gam.head(2)

,genres,app_name,tags,specs,developer
id,,,,,
761140,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],Kotoshiro
670290,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Poolians.com


Agrego la calificación promedio de los juegos y la cantidad de veces que fueron puntuados.



In [39]:
#Calculo la media de los juegos calificados y la cantidad de veces guandandolo en game_vote
groupe = data_Rev.groupby('product_id')
game_vote = pd.DataFrame(groupe['calification'].agg(np.mean))
game_vote.rename(columns={'calification':'vote_average'}, inplace=True) 
game_vote['vote_count'] = data_Rev['product_id'].value_counts()
game_vote.head(2)

,vote_average,vote_count
product_id,,
10,2.342857,175
20,1.808783,1093


In [40]:
#Como trabajo con los datasets incompletos, filtro y dejo solo los juegos que tengo votos
data = data_Gam[data_Gam.index.isin(game_vote.index)]
data.head()

,genres,app_name,tags,specs,developer
id,,,,,
670290,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Poolians.com
765320,"[Casual, Indie, Simulation]",Planetarium 2 - Zen Odyssey,"[Indie, Casual, Simulation]",[Single-player],Ghulam Jewel
70,[Action],Half-Life,"[FPS, Classic, Action, Sci-fi, Singleplayer, S...","[Single-player, Multi-player, Valve Anti-Cheat...",Valve
716110,"[Adventure, Casual, Indie, RPG]",Bitcoin Clicker,"[Adventure, RPG, Indie, Casual, Clicker]","[Single-player, Steam Achievements, Partial Co...","lalalaZero,Urbanoff"
1630,[Strategy],Disciples II: Rise of the Elves,"[Strategy, Turn-Based Strategy, Fantasy, Turn-...","[Single-player, Multi-player, Co-op]",Strategy First


In [41]:
#Agrego los votos y calificaciones a los juegos
data['vote_count'] = data.index.to_series().map(game_vote['vote_count'])
data['vote_average'] = data.index.to_series().map(game_vote['vote_average'])

In [42]:
data.head(2)

,genres,app_name,tags,specs,developer,vote_count,vote_average
id,,,,,,,
670290,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Poolians.com,15,1.0
765320,"[Casual, Indie, Simulation]",Planetarium 2 - Zen Odyssey,"[Indie, Casual, Simulation]",[Single-player],Ghulam Jewel,2,1.0


## Recomendador Simple

Utilizo las calificaciones de TMDB para crear nuestra **Tabla de juegos principales.** Usaré la fórmula de *calificación ponderada* de IMDB para construir mi tabla. Matemáticamente, se representa de la siguiente manera:

Calificación ponderada (WR) = $(\frac{v}{v + m} . R) + (\frac{m}{v + m} . C)$

* *v* es el número de votos para el juego.
* *m* son los votos mínimos requeridos para aparecer en la tabla.
* *R* es la calificación promedio del juego.
* *C* es el voto medio de todo el informe.

El siguiente paso es determinar un valor apropiado para *m*, los votos mínimos requeridos para aparecer en la tabla. Usaremos **percentil 95** como nuestro límite. En otras palabras, para que un juego aparezca en las listas, debe tener más votos que al menos el 95% de los juegos de la lista.


In [43]:
vote_counts = data[data['vote_count'].notnull()]['vote_count'].astype('int')
vote_averages = data[data['vote_average'].notnull()]['vote_average'].astype('float64')
C = vote_averages.mean()
C

1.5692689344336095

In [44]:
m = vote_counts.quantile(0.95)
m

958.0

In [45]:
qualified = data[(data['vote_count'] >= m) & (data['vote_count'].notnull()) & (data['vote_average'].notnull())][['app_name', 'vote_count', 'vote_average','genres']]
qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('float64')
qualified.shape

(374, 4)

In [46]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [47]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [48]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

In [49]:
qualified.head(10)

,app_name,vote_count,vote_average,genres,wr
id,,,,,
48700,Mount & Blade: Warband,18348,3.713974,"[Action, RPG]",3.607550
8930,Sid Meier's Civilization® V,18676,3.676108,[Strategy],3.573310
440,Team Fortress 2,91128,3.576091,"[Action, Free to Play]",3.555213
4000,Garry's Mod,24141,3.559546,"[Indie, Simulation]",3.483579
107410,Arma 3,20172,3.570444,"[Action, Simulation, Strategy]",3.479714
427520,Factorio,10478,3.636572,"[Casual, Indie, Simulation, Strategy, Early Ac...",3.463393
252490,Rust,51080,3.494695,"[Action, Adventure, Indie, Massively Multiplay...",3.459248
230410,Warframe,27199,3.524174,"[Action, Free to Play]",3.457661
220200,Kerbal Space Program,16607,3.561691,"[Indie, Simulation]",3.453024


<img src="https://cdn.akamai.steamstatic.com/steam/apps/48700/ss_90c40b92912c02e8189d7eec53e4152dac97262b.600x338.jpg?t=1589227310" width="200" height="200">

<img src="https://cdn.akamai.steamstatic.com/steam/apps/8930/ss_e6f9ef54f06ffebc9445d8b29d8d2054fa3185d4.600x338.jpg?t=1579731804" width="200" height="200">

<img src="https://cdn.akamai.steamstatic.com/steam/apps/440/0000002574.600x338.jpg?t=1592263852" width="200" height="200">


Podemos ver la eleccion de nuestro recomendador simple, los generos de los juegos son variados, hay un sesgo hacia los juegos de accion (elección de la mayoria de los juagdores), por ser tan simple obtenemos muy buenos resultados.

In [50]:
#Guardo
if True:
    data_Gam.to_csv(r'D:/Acamica/Clases Acamica/Sprint 3/Proyecto/Datos_2_Games.csv', index= True)   

## Filtro Basado en Contenido 

Para personalizar nuestras recomendaciones, voy a preparar un motor que calcule la similitud entre juegos en función de ciertas métricas y sugiera juegos que son más similares a un juego en particular que le gustó a un usuario.
Comienzo con la construcción de un recomendador usando 'genres','tags', 'specs' y 'developer'. No tenemos una métrica cuantitativa para juzgar el rendimiento de nuestra máquina, por lo que esto tendrá que hacerse cualitativamente.


In [51]:
data_Gam.head(2)

,genres,app_name,tags,specs,developer
id,,,,,
761140,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],Kotoshiro
670290,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Poolians.com


In [52]:
#Relleno los Nan con espacios vacios []
data_Gam['genres'] = data_Gam['genres'].fillna('[]')
data_Gam['tags'] = data_Gam['tags'].fillna('[]')
data_Gam['specs'] = data_Gam['specs'].fillna('[]')
data_Gam['developer'] = data_Gam['developer'].fillna('[]')

In [56]:
q = data_Gam.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
i = data_Gam.apply(lambda x: pd.Series(x['tags']),axis=1).stack().reset_index(level=1, drop=True)
o = data_Gam.apply(lambda x: pd.Series(x['specs']),axis=1).stack().reset_index(level=1, drop=True)
q = q.append(i)
q = q.append(o)
q = q.append(data_Gam['developer'])

In [57]:
q.shape

(210969,)

In [58]:
q = q.value_counts()
q[:5]

Indie            16867
Single-player    13800
Action           12178
Casual            9050
Adventure         8886
dtype: int64

In [59]:
q = q[q > 1]

In [60]:
q.shape

(2306,)

In [79]:
#implemento stemmer para reducir una palabra a su raíz 
stemmer = SnowballStemmer('english')

In [62]:
#Filtro de palabras 
def filter_keywords(x):
    words = []
    for i in x:
        if i in q:
            words.append(i)
    return words

In [63]:
data_Gam['genres'] = data_Gam['genres'].apply(filter_keywords)
data_Gam['genres'] = data_Gam['genres'].apply(lambda x: [stemmer.stem(i) for i in x])
data_Gam['genres'] = data_Gam['genres'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
data_Gam['tags'] = data_Gam['tags'].apply(filter_keywords)
data_Gam['tags'] = data_Gam['tags'].apply(lambda x: [stemmer.stem(i) for i in x])
data_Gam['tags'] = data_Gam['tags'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
data_Gam['specs'] = data_Gam['specs'].apply(filter_keywords)
data_Gam['specs'] = data_Gam['specs'].apply(lambda x: [stemmer.stem(i) for i in x])
data_Gam['specs'] = data_Gam['specs'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
data_Gam['developer'] = data_Gam['developer'].apply(lambda x: str.lower(x.replace(" ", "")))
data_Gam['developer'] = data_Gam['developer'].apply(lambda x: [stemmer.stem(x)])

In [64]:
data_Gam['soup'] = data_Gam['specs'] + data_Gam['tags']  + data_Gam['genres'] + data_Gam['developer']
data_Gam['soup'] = data_Gam['soup'].apply(lambda x: ' '.join(x))

In [65]:
data_Gam.head(2)

,genres,app_name,tags,specs,developer,soup
id,,,,,,
761140,"[action, casual, indi, simul, strategi]",Lost Summoner Kitty,"[strategi, action, indi, casual, simul]",[single-play],[kotoshiro],single-play strategi action indi casual simul ...
670290,"[casual, freetoplay, indi, simul, sport]",Real Pool 3D - Poolians,"[freetoplay, simul, sport, casual, indi, multi...","[single-play, multi-play, onlinemulti-play, in...",[poolians.com],single-play multi-play onlinemulti-play in-app...


In [68]:
data_Gam.soup[761140]

'single-play strategi action indi casual simul action casual indi simul strategi kotoshiro'

In [69]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(data_Gam['soup'])
count_matrix.shape

(16067, 32504)

#### Cosine Similarity

Usaré la similitud de coseno para calcular una cantidad numérica que denote la similitud entre dos juegos. Matemáticamente, se define de la siguiente manera:

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $

In [70]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim[0]

array([1.        , 0.41811291, 0.38769906, ..., 0.70618786, 0.39036003,
       0.27322953])

In [71]:
data_Gam = data_Gam.reset_index()
titles = data_Gam['app_name']
indices = pd.Series(data_Gam.index, index=data_Gam['app_name'])

In [72]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    games_indices = [i[0] for i in sim_scores] 
    return titles.iloc[games_indices]

In [81]:
get_recommendations('Counter-Strike').head(10)

16053                               Deathmatch Classic
16005                           Counter-Strike: Source
16056                                    Day of Defeat
15720                            Day of Defeat: Source
45       QUAKE Mission Pack 2: Dissolution of Eternity
15889               INSURGENCY: Modern Infantry Combat
14671                            Call of Duty®: Ghosts
172                   Call of Duty®: Modern Warfare® 2
14172                             Half-Life Soundtrack
15896          Tom Clancy's Ghost Recon® Desert Siege™
Name: app_name, dtype: object

<img src="https://cdn.akamai.steamstatic.com/steam/apps/10/0000000136.600x338.jpg?t=1602535893" width="200" height="200">

<img src="https://cdn.akamai.steamstatic.com/steam/apps/40/0000000142.600x338.jpg?t=1568752159" width="200" height="200">

<img src="https://cdn.akamai.steamstatic.com/steam/apps/30/0000000171.600x338.jpg?t=1512413490" width="200" height="200">



In [76]:
get_recommendations('MotoGP™17').head(10)

10557                               Ride 2
8743              Ride 2 Free Bikes Pack 9
11327             Valentino Rossi The Game
4159              Ride 2 Free Bikes Pack 2
4241              Ride 2 Free Bikes Pack 3
4442            Ride 2 2017 Top Bikes Pack
9546              Ride 2 Free Bikes Pack 4
10376    Ride 2 Limited Edition Bikes Pack
12322                    MotoGP™15 Compact
9850          Ride 2 Rising Sun Bikes Pack
Name: app_name, dtype: object

<img src="https://cdn.akamai.steamstatic.com/steam/apps/561610/ss_3e8835f1d35495a1066772c40c4e221ca3e5b7a6.600x338.jpg?t=1576506007" width="200" height="200">

<img src="https://cdn.akamai.steamstatic.com/steam/apps/477770/ss_0bcc17a32d71edce4fc72d0b4318e21243eaefb8.600x338.jpg?t=1576507292" width="200" height="200">

<img src="https://cdn.akamai.steamstatic.com/steam/apps/438430/ss_c5db82e2f0eb4772eb89cc0133caec6a4365a768.600x338.jpg?t=1574434309" width="200" height="200">



Estoy mucho más satisfecho con los resultados que obtengo esta vez. Las recomendaciones parecen haber reconocido otros juegos similares (debido al peso que se le dio genero y tags ) y ponerlas como recomendaciones principales. Por supuesto, podemos experimentar con este motor probando diferentes pesos para nuestras funciones (genereos, tags, developer y specs), limitando la cantidad de palabras clave que se pueden usar en la sopa, sopesando los géneros en función de su frecuencia, mostrando solo juegos con el mismo genero, etc.

## Filtro Colaborativo 

Nuestro motor basado en contenido adolece de graves limitaciones. Solo es capaz de sugerir juegos *cercanos* a un juego determinado. Es decir, no es capaz de capturar los gustos y brindar recomendaciones de todos los géneros.

Además, el motor que construimos no es realmente personal, ya que no captura los gustos y prejuicios personales de un usuario. Cualquiera que consulte nuestro motor para obtener recomendaciones basadas en un juego recibirá las mismas recomendaciones para ese juego, independientemente de quién sea.

Por lo tanto, en esta sección, utilizaremos una técnica llamada **Filtrado colaborativo** para hacer recomendaciones a los jugadores. El filtrado colaborativo se basa en la idea de que los usuarios similares a mí pueden usarse para predecir cuánto me gustará un producto o servicio en particular que esos usuarios han experimentado pero yo no.

No implementaré el filtrado colaborativo desde cero. En su lugar, usaré la biblioteca **Surprise** que usó algoritmos extremadamente poderosos como **Descomposición de valores singulares (SVD)** para minimizar el RMSE (Error cuadrático medio) y brindar excelentes recomendaciones.

**Elimino los usuarios que solo probaron un juego para disminuir el problema del arranque en frio**

In [83]:
data_Rev = pd.read_csv(r'D:/Acamica/Clases Acamica/Sprint 3/Proyecto/Datos_2_Review.csv')

In [84]:
data_Rev.head(2)

,username,calification,product_id
0,Chaos Syren,1,725280
1,hello?<,2,328100


In [85]:
cold = pd.DataFrame(data_Rev['username'].value_counts()>1)
cold.head(2)

,username
123,True
Alex,True


In [86]:
data_Rev = data_Rev.set_index('username')
data_Rev['cold'] = data_Rev.index.to_series().map(cold['username'])
data_Rev['username'] = data_Rev.index
data_Rev.reset_index(drop = True,inplace = True)

In [87]:
data_Rev.shape

(3875815, 4)

In [88]:
data_r = data_Rev[data_Rev['cold']==True]
data_r.shape

(2802833, 4)

In [89]:
data_Rev.drop('cold',1,inplace=True)

### SVD

Realizo un modelo simple sin parametros para utilizar de benchmark

In [11]:
reader = Reader()

In [12]:
df = Dataset.load_from_df(data_r[['username', 'product_id', 'calification']], reader)

In [16]:
#Divido en train y test
trainset, testset = train_test_split(df, test_size=.25)

In [2]:
algo = SVD()

In [147]:
#entreno
algo.fit(trainset)

In [148]:
#realizo las predicciones 
predictions = algo.test(testset)

In [149]:
accuracy.rmse(predictions)

RMSE: 0.8445


0.8444568833070987

Optenemos un RMSE 4% mejor que el proyecto 3 y sin optimizar 

### Optimizando los hiperparametros

preparo una lista de parametros para optimizar el modelo

In [13]:
param_grid = {'n_factors': [4,8,15,16],'n_epochs': [15,18,20,23], 'lr_all': [0.002,0.05,0.010],
              'reg_all': [0.01,0.02,0.002,0.4]}

gs = GridSearchCV(SVD, param_grid = param_grid, measures=['rmse'], cv=2, n_jobs = -1)

gs.fit(df)

In [14]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8504393325873595
{'n_factors': 4, 'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.02}


In [17]:
algo_opt = SVD(n_factors=4, n_epochs=15, lr_all=0.01,reg_all=0.02)
algo_opt.fit(trainset)

In [18]:
pred = algo_opt.test(testset)

In [19]:
accuracy.rmse(pred)

RMSE: 0.8418


0.8417587949873373

Mejoro en un 0.31% respecto al modelo no optimizado

## Recomendador Híbrido

In [90]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [91]:
id_map = pd.read_csv(r'D:/Acamica/Clases Acamica/Sprint 3/Proyecto/Datos_2_Games.csv')[['id', 'app_name']]
id_map['id'] = id_map['id'].apply(convert_int)
id_map['g_id'] = id_map.index
id_map = id_map.set_index('app_name')

In [92]:
indices_map = id_map.set_index('g_id')

In [93]:
data_Gam['id'] = data_Gam.index
data_Gam.reset_index(drop = True,inplace = True)

In [122]:
def hybrid(username, title):
    idx = indices[title]
#    print(idx)
    Id = id_map.loc[title]['id']
    app_name = id_map.loc[title]['g_id']
    
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:150]
    game_indices = [i[0] for i in sim_scores]
    
    games = data_Gam.iloc[game_indices][['app_name','id']]
    games['g_id'] = games.index
    games['est'] = games['g_id'].apply(lambda x: algo_opt.predict(username, indices_map.loc[x]['id']).est)
    games = games.sort_values('est', ascending=False)
    games.drop(['id','g_id'],1,inplace = True)
    
    return games.head(10)

In [125]:
hybrid('Nick','Call of Duty®: Ghosts')

,app_name,est
15003,Arma 3,3.408192
1619,DRAGON BALL XENOVERSE,3.120793
2373,Call of Duty®: Black Ops III,3.095888
3993,Killing Floor 2,2.929810
15876,Left 4 Dead,2.541631
273,Call of Duty®: Black Ops,2.453846
3038,Battleborn,2.448131
933,Insurgency,2.302985
15146,Aliens: Colonial Marines - Bug Hunt DLC,2.207042
693,Aliens: Colonial Marines Sawed-off Double Barr...,2.207042


In [126]:
hybrid('Tuong','Call of Duty®: Ghosts')

,app_name,est
2373,Call of Duty®: Black Ops III,5.000000
1619,DRAGON BALL XENOVERSE,5.000000
3993,Killing Floor 2,5.000000
15003,Arma 3,5.000000
3038,Battleborn,4.828454
15876,Left 4 Dead,4.626798
273,Call of Duty®: Black Ops,4.604869
16056,Day of Defeat,4.564539
2454,HELLDIVERS™ - Vehicles Pack,4.404229
2452,HELLDIVERS™ - Terrain Specialist Pack,4.404229


Vemos que para nuestro recomendador híbrido, obtenemos diferentes recomendaciones para diferentes usuarios, aunque el juego es el mismo. Por lo tanto, nuestras recomendaciones son más personalizadas y adaptadas a usuarios particulares.

## Conclusión 

En este proyecto, he construido 4 motores de recomendación diferentes basados en diferentes ideas y algoritmos. Son los siguientes:

1. **Recomendador simple:** Este sistema utilizó el recuento de votos y los promedios de votos de TMDB generales para crear gráficos de películas principales, en general y para un género específico. Se utilizó el Sistema de Calificación Ponderada IMDB para calcular las calificaciones en las que finalmente se realizó la clasificación.
2. **Recomendador basado en contenido:** Creamos un motor basado en contenido; que tomó los generos de los juegos, los tags, los specs y los desarrolladores para generar predicciones. 
3. **Filtrado colaborativo:** Usamos la poderosa Biblioteca Surprise para crear un filtro colaborativo basado en la descomposición de un solo valor. El RMSE obtenido fue inferior a 1 y el motor dio calificaciones estimadas para un usuario y un juego determinado.
4. **Motor híbrido:** Reunimos ideas de contenido y filtrado colaborativo para crear un motor que ofreciera sugerencias de películas a un usuario en particular en función de las calificaciones estimadas que había calculado internamente para ese usuario.